# Odpowiedzi na zadania laboratoryjne

## Zadanie 1

In [ ]:
# Zestaw 1 zad.1

import numpy as np
class SimpleNeuralNetwork:

    def __init__(self):
        np.random.seed(1)
        self.weights = 2 * np.random.random((3, 1)) - 1
 
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def d_sigmoid(self, x):
        return x * (1 - x)

    def train(self, train_input, train_output, train_iters):
        for _ in range(train_iters):
            propagation_result = self.propagation(train_input)
            self.backward_propagation(
                propagation_result, train_input, train_output)

    def propagation(self, inputs):
        """
        Propagation process
        """
        return self.sigmoid(np.dot(inputs.astype(float), self.weights))

    def backward_propagation(self, propagation_result, train_input, train_output):
        """
        Backward propagation process 
        """
        error = train_output - propagation_result
        self.weights += np.dot(
            train_input.T, error * self.d_sigmoid(propagation_result)
        )
 
network = SimpleNeuralNetwork()

print(network.weights)

train_inputs = np.array(
    [[1, 1, 0], [1, 1, 1], [1, 1, 0], [1, 0, 0], [0, 1, 1], [0, 1, 0], ]
                       )

train_outputs = np.array([[0, 1, 0, 0, 1, 0]]).T

train_iterations = 50000

network.train(train_inputs, train_outputs, train_iterations)

print(network.weights)

test_data = np.array([[1, 1, 1], [1, 0, 0], [0, 1, 1], [0, 1, 0], ])

for data in test_data:
    print(f"Wynikiem dla danych {data} jest:")
    print(network.propagation(data))


## Zadanie 2

In [ ]:
# Zestaw 1-2 zad.2
import pandas as pd
import numpy as np

np.set_printoptions(suppress=True)

CarPricesDataNumeric=pd.read_pickle('CarPricesData.pkl')
CarPricesDataNumeric.head()

TargetVariable=['Price']
Predictors=['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors']

X = CarPricesDataNumeric[Predictors].values
y = CarPricesDataNumeric[TargetVariable].values
 
from sklearn.preprocessing import StandardScaler
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y)

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

from keras.models import Sequential
from keras.layers import Dense

#ANN model
model = Sequential()

# the Input layer and first hidden layer
model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))

#the Second layer of the model
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')

# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

# Tuning of parameters
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    # Defining the list of hyper parameters to try
    batch_size_list = [5, 10, 15, 20]
    epoch_list = [5, 10, 50, 100]

    SearchResultsData = pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])

    # initializing the trials
    TrialNumber = 0

    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber += 1

            # create ANN model
            model = Sequential()

            # the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))

            # the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))

            # The output neuron is a single fully connected node 
            model.add(Dense(1, kernel_initializer='normal'))

            model.compile(loss='mean_squared_error', optimizer='adam')

            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))

            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', 100-MAPE)

            SearchResultsData=SearchResultsData.append(pd.DataFrame(
                data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                columns=['TrialNumber', 'Parameters', 'Accuracy'])
                                                      )

    return(SearchResultsData)

ResultsData = FunctionFindBestParams(X_train, y_train, X_test, y_test)

from keras.models import Sequential
from keras.layers import Dense

#Deep ANN model 
def make_regression_ann(Optimizer_trial):
    model = Sequential()
    model.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=Optimizer_trial)
    return model

# Fitting the ANN to the Training set
model.fit(X_train, y_train ,batch_size = 15, epochs = 5, verbose=0)

Predictions = model.predict(X_test)

# Scaling the predicted Price data back to original price scale
Predictions = TargetVarScalerFit.inverse_transform(Predictions)

# Scaling the y_test Price data back to original price scale
y_test_orig = TargetVarScalerFit.inverse_transform(y_test)

print("########## Razem: ", round((EndTime-StartTime)/60), 'minut')

grid_search.best_params_

# Scaling the test data back to original scale
Test_Data = PredictorScalerFit.inverse_transform(X_test)

TestingData = pd.DataFrame(data=Test_Data, columns=Predictors)
TestingData['Price'] = y_test_orig
TestingData['PredictedPrice'] = Predictions

TestingData.head()

# APE
APE = 100*(abs(TestingData['Price']-TestingData['PredictedPrice'])/TestingData['Price'])

TestingData['APE'] = APE

print('The Accuracy of ANN model is:', 100-np.mean(APE))

TestingData.head()

###########################################

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor

Parameter_Trials = {'batch_size':[10,20,30],
                      'epochs':[10,20],
                    'Optimizer_trial':['adam', 'rmsprop']
                   }

#ANN model
RegModel = KerasRegressor(make_regression_ann, verbose=0)

from sklearn.metrics import make_scorer

def Accuracy_Score(orig,pred):
    MAPE = np.mean(100 * (np.abs(orig-pred)/orig))
    print('#'*70,'Accuracy:', 100-MAPE)
    return(100-MAPE)

custom_Scoring = make_scorer(Accuracy_Score, greater_is_better=True)

#the Grid search space
grid_search = GridSearchCV(estimator=RegModel, 
                         param_grid=Parameter_Trials, 
                         scoring=custom_Scoring, 
                         cv=5)

import time
StartTime = time.time()

grid_search.fit(X,y, verbose=1)

EndTime=time.time()

## Zadanie 3

In [ ]:
# Zestaw 1-2 zad.3

import tensorflow as tf

# wczytanie danych 
fashion_mnist = tf.keras.datasets.fashion_mnist

(X_train, y_train), (X_val, y_val) = fashion_mnist.load_data()

print(f'Zbiór uczący: {X_train.shape}, zbiór walidacyjny: {X_val.shape}')

import matplotlib.pyplot as plt

plt.figure(figsize=(7,7))
plt.imshow(X_train[0], cmap=plt.cm.binary)
plt.colorbar()
plt.show()

def plot_digit(digit, dem=28, font_size=8):
    max_ax = font_size * dem

    fig = plt.figure(figsize=(10,10))
    plt.xlim([0, max_ax])
    plt.ylim([0, max_ax])
    plt.axis('off')

    for idx in range(dem):
        for jdx in range(dem):
            t = plt.text(idx*font_size, max_ax - jdx*font_size, 
                         digit[jdx][idx], fontsize=font_size, 
                         color="#000000")
            c = digit[jdx][idx] / 255.
            t.set_bbox(dict(facecolor=(c, c, c), alpha=0.5, 
                            edgecolor='#f1f1f1'))
            
    plt.show()

plot_digit(X_train[0])
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

plt.figure(figsize=(14,10))
for i in range(40):
    plt.subplot(5, 8, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

from tensorflow.keras.utils import to_categorical

X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
y_train = to_categorical(y_train, len(class_names))
y_val = to_categorical(y_val, len(class_names))

from tensorflow.keras.models import Sequential

model = Sequential()

from tensorflow.keras.layers import Flatten, add

model.add(Flatten(input_shape=(28, 28)))

from tensorflow.keras.layers import Dense

model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation = 'softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

history = model.fit(X_train, 
                    y_train, 
                    epochs=10,
                    verbose=1,
                    batch_size = 256,
                    validation_data = (X_val, y_val)
                   )

def draw_curves(history, key1='accuracy', ylim1=(0.8, 1.00), 
                key2='loss', ylim2=(0.0, 1.0)):
    plt.figure(figsize=(12,4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history[key1], "r--")
    plt.plot(history.history['val_' + key1], "g--")
    plt.ylabel(key1)
    plt.xlabel('Epoch')
    plt.ylim(ylim1)
    plt.legend(['train', 'test'], loc='best')
    plt.subplot(1, 2, 2)
    plt.plot(history.history[key2], "r--")
    plt.plot(history.history['val_' + key2], "g--")
    plt.ylabel(key2)
    plt.xlabel('Epoch')
    plt.ylim(ylim2)
    plt.legend(['train', 'test'], loc='best')
    plt.show()

draw_curves(history, key1='accuracy', ylim1=(0.7, 0.95), 
            key2='loss', ylim2=(0.0, 0.8))    

## Zadanie 5

In [ ]:
# zad.5 - najprościej

import keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler

# Wersja bez tuningu! Polecam przetestować inne opcje - AT

seed = 2024
numpy.random.seed(seed)

dataset = numpy.loadtxt('pima-indians-diabetes.csv', delimiter=",")

X = dataset[:,0:8]  
Y = dataset[:,8]     
scaler = StandardScaler()
X = scaler.fit_transform(X)

model = Sequential()

model.add(Dense(500, activation='relu')) 
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X, Y, epochs=150, batch_size=10,  verbose=2) # 150 epoch, 10 batch size, verbose = 2

scores = model.evaluate(X, Y)

print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

predictions = model.predict(X)    

print(predictions)
